<a href="https://colab.research.google.com/github/thedavidneufeld/ResearchCode/blob/main/RQI_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [1]:
import numpy as np

**Helper Functions**

In [2]:
# given an approximate eigenvalue (mu) and a matrix M
# returns (mu)*I-M
def A(mu, M):
    I = np.eye(M.shape[0])
    return mu*I-M

# calculates and returns the conditon number of M
# based on sigma_1/sigma_n
def svd_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-1]

# calculates and returns sigma_1/sigma_{n-1}
def svd_desired_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-2]

# calculates a preconditioned matrix C based on
# an approximate eigenvalue (mu), a matrix M, and vectors
# y and v
# returns C
def f(mu, M, y, v):
    # convert y and v to column vectors
    Y = y[:, np.newaxis]
    V = v[:, np.newaxis]
    # calculate (mu)*I-M
    A1 = A(mu, M)
    # calculate ((mu)*I-M)/||((mu)*I-M)||+YV^H
    C = A1/np.linalg.norm(A1, 2) + np.dot(Y, V.T.conj())
    # return C
    return C

**RQI Algorithm**

In [3]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
def RQI(M, mu, ep, it):
    # create a random vector with norm of 1
    # vector needs to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    # track condition number of (mu)I-M through iterations
    conds = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate the condition number of (mu)I-M
        conds = np.append(conds, svd_cond(muI_M))
        # solve the system ((mu)I-M)x=y
        x = np.linalg.solve(muI_M, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (x^T)Mx
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x, 2) <= ep*np.linalg.norm(M, 2):
            return mu, y, i, conds
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds


**RQI Algorithm with Additive Preconditioning**

In [4]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
#
# similar to regular RQI, but uses additive preprocessing
def RQIAPP(M, mu, ep, it):
    # create two random vectors with norm of 1
    # vectors need to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    v = np.random.rand(M.shape[0])
    v = v/np.linalg.norm(v)
    # track condition number of (mu)I-M, C, and the desired condition number
    conds = np.empty(0)
    conds_C = np.empty(0)
    conds_desired = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate C
        C = f(mu, M, y, v)
        # calculate the condition numbers
        conds = np.append(conds, svd_cond(muI_M))
        conds_C = np.append(conds_C, svd_cond(C))
        conds_desired = np.append(conds_desired, svd_desired_cond(muI_M))
        # solve the system Cx=y
        x = np.linalg.solve(C, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (y^T)My
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x, 2) <= ep*np.linalg.norm(M, 2):
            return mu, y, i, conds, conds_C, conds_desired
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds, conds_C, conds_desired

**Test and Compare Algorithms**

In [5]:
# Setup

# create a random matrix M
M = np.random.rand(100, 100)
# calculate the eigenvalues/eigenvectors of M
vals, vecs = np.linalg.eig(M)
# extract the first eigenvalue of M and subtract
# a small value to make it an approximation
mu = vals[0]-0.25

In [6]:
# Print condition number of M
svd_cond(M)

6035.002098743229

In [7]:
# Test RQI with 1e-4
result = RQI(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (49.70635339423618+0j)
Eigenvector: [-0.09318887+0.j -0.10255637+0.j -0.11091952+0.j -0.10177865+0.j
 -0.09711909+0.j -0.10651015+0.j -0.10864416+0.j -0.09964547+0.j
 -0.09689994+0.j -0.09505389+0.j -0.09305672+0.j -0.10299191+0.j
 -0.09589768+0.j -0.10137648+0.j -0.09868125+0.j -0.10214183+0.j
 -0.09932728+0.j -0.10352883+0.j -0.10850548+0.j -0.09190121+0.j
 -0.09411318+0.j -0.10033302+0.j -0.10221065+0.j -0.10409916+0.j
 -0.09758906+0.j -0.0960253 +0.j -0.10927716+0.j -0.0941929 +0.j
 -0.10094037+0.j -0.08709107+0.j -0.09618714+0.j -0.10648279+0.j
 -0.10416788+0.j -0.09638744+0.j -0.1006612 +0.j -0.09329455+0.j
 -0.10323212+0.j -0.09863298+0.j -0.09548265+0.j -0.10228473+0.j
 -0.09859791+0.j -0.09996165+0.j -0.10730828+0.j -0.1009598 +0.j
 -0.09144818+0.j -0.10664134+0.j -0.10529857+0.j -0.09590143+0.j
 -0.10938656+0.j -0.09563107+0.j -0.09183601+0.j -0.1019908 +0.j
 -0.10619515+0.j -0.09898924+0.j -0.09721324+0.j -0.10380735+0.j
 -0.09522432+0.j -0.10509251+0.j -0.104989

In [8]:
# Test RQIAPP with 1e-4
result = RQIAPP(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers for (mu)I-M:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.70635350817334+0j)
Eigenvector: [0.09318889+0.j 0.10255637+0.j 0.11091952+0.j 0.10177867+0.j
 0.0971191 +0.j 0.10651014+0.j 0.10864417+0.j 0.09964548+0.j
 0.09689994+0.j 0.09505389+0.j 0.09305671+0.j 0.10299194+0.j
 0.0958977 +0.j 0.10137647+0.j 0.09868123+0.j 0.10214183+0.j
 0.0993273 +0.j 0.10352886+0.j 0.10850548+0.j 0.09190123+0.j
 0.0941132 +0.j 0.10033304+0.j 0.10221065+0.j 0.10409917+0.j
 0.09758908+0.j 0.09602528+0.j 0.10927718+0.j 0.09419291+0.j
 0.10094038+0.j 0.08709106+0.j 0.09618714+0.j 0.10648277+0.j
 0.10416787+0.j 0.09638743+0.j 0.10066121+0.j 0.09329455+0.j
 0.10323212+0.j 0.098633  +0.j 0.09548265+0.j 0.10228473+0.j
 0.09859789+0.j 0.09996166+0.j 0.10730827+0.j 0.10095981+0.j
 0.09144819+0.j 0.10664133+0.j 0.10529855+0.j 0.09590143+0.j
 0.10938656+0.j 0.09563107+0.j 0.091836  +0.j 0.1019908 +0.j
 0.10619514+0.j 0.09898924+0.j 0.09721324+0.j 0.10380736+0.j
 0.09522429+0.j 0.1050925 +0.j 0.10498975+0.j 0.09955274+0.j
 0.10134655+0.j 0.10516052+0.j 0.0976

In [9]:
# Test RQI with 1e-6
result = RQI(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (49.706353387090914+0j)
Eigenvector: [-0.09318887+0.j -0.10255637+0.j -0.11091952+0.j -0.10177865+0.j
 -0.09711908+0.j -0.10651015+0.j -0.10864416+0.j -0.09964546+0.j
 -0.09689994+0.j -0.09505389+0.j -0.09305672+0.j -0.10299192+0.j
 -0.09589768+0.j -0.10137649+0.j -0.09868125+0.j -0.10214183+0.j
 -0.09932728+0.j -0.10352883+0.j -0.10850549+0.j -0.09190122+0.j
 -0.09411318+0.j -0.10033302+0.j -0.10221064+0.j -0.10409916+0.j
 -0.09758907+0.j -0.0960253 +0.j -0.10927716+0.j -0.0941929 +0.j
 -0.10094037+0.j -0.08709107+0.j -0.09618714+0.j -0.10648278+0.j
 -0.10416788+0.j -0.09638744+0.j -0.1006612 +0.j -0.09329455+0.j
 -0.10323212+0.j -0.09863298+0.j -0.09548265+0.j -0.10228473+0.j
 -0.09859791+0.j -0.09996165+0.j -0.10730829+0.j -0.1009598 +0.j
 -0.09144817+0.j -0.10664134+0.j -0.10529857+0.j -0.09590143+0.j
 -0.10938656+0.j -0.09563107+0.j -0.09183601+0.j -0.10199079+0.j
 -0.10619515+0.j -0.09898924+0.j -0.09721324+0.j -0.10380735+0.j
 -0.09522432+0.j -0.10509251+0.j -0.10498

In [10]:
# Test RQIAPP with 1e-6
result = RQIAPP(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.70635338611922+0j)
Eigenvector: [0.09318887+0.j 0.10255637+0.j 0.11091953+0.j 0.10177865+0.j
 0.09711908+0.j 0.10651015+0.j 0.10864416+0.j 0.09964547+0.j
 0.09689994+0.j 0.09505389+0.j 0.09305672+0.j 0.10299192+0.j
 0.09589768+0.j 0.10137649+0.j 0.09868125+0.j 0.10214183+0.j
 0.09932728+0.j 0.10352883+0.j 0.10850549+0.j 0.09190121+0.j
 0.09411318+0.j 0.10033302+0.j 0.10221064+0.j 0.10409916+0.j
 0.09758907+0.j 0.0960253 +0.j 0.10927716+0.j 0.0941929 +0.j
 0.10094037+0.j 0.08709107+0.j 0.09618714+0.j 0.10648279+0.j
 0.10416788+0.j 0.09638744+0.j 0.1006612 +0.j 0.09329455+0.j
 0.10323212+0.j 0.09863298+0.j 0.09548265+0.j 0.10228473+0.j
 0.09859791+0.j 0.09996165+0.j 0.10730829+0.j 0.1009598 +0.j
 0.09144817+0.j 0.10664134+0.j 0.10529858+0.j 0.09590143+0.j
 0.10938656+0.j 0.09563107+0.j 0.09183601+0.j 0.1019908 +0.j
 0.10619515+0.j 0.09898924+0.j 0.09721324+0.j 0.10380735+0.j
 0.09522432+0.j 0.10509251+0.j 0.10498977+0.j 0.09955273+0.j
 0.10134654+0.j 0.10516052+0.j 0.0976

In [11]:
# Test RQI with 1e-8
result = RQI(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: None
Eigenvector: None
Iterations: 100
Condition Numbers: [2.14221278e+02 1.24370676e+05 9.16076653e+10 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15 5.77023632e+15 5.77023632e+15 5.77023632e+15
 5.77023632e+15

In [12]:
# Test RQIAPP with 1e-8
result = RQIAPP(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.7063533870909+0j)
Eigenvector: [0.09318887+0.j 0.10255637+0.j 0.11091952+0.j 0.10177865+0.j
 0.09711908+0.j 0.10651015+0.j 0.10864416+0.j 0.09964546+0.j
 0.09689994+0.j 0.09505389+0.j 0.09305672+0.j 0.10299192+0.j
 0.09589768+0.j 0.10137649+0.j 0.09868125+0.j 0.10214183+0.j
 0.09932728+0.j 0.10352883+0.j 0.10850549+0.j 0.09190122+0.j
 0.09411318+0.j 0.10033302+0.j 0.10221064+0.j 0.10409916+0.j
 0.09758907+0.j 0.0960253 +0.j 0.10927716+0.j 0.0941929 +0.j
 0.10094037+0.j 0.08709107+0.j 0.09618714+0.j 0.10648278+0.j
 0.10416788+0.j 0.09638744+0.j 0.1006612 +0.j 0.09329455+0.j
 0.10323212+0.j 0.09863298+0.j 0.09548265+0.j 0.10228473+0.j
 0.09859791+0.j 0.09996165+0.j 0.10730829+0.j 0.1009598 +0.j
 0.09144817+0.j 0.10664134+0.j 0.10529857+0.j 0.09590143+0.j
 0.10938656+0.j 0.09563107+0.j 0.09183601+0.j 0.10199079+0.j
 0.10619515+0.j 0.09898924+0.j 0.09721324+0.j 0.10380735+0.j
 0.09522432+0.j 0.10509251+0.j 0.10498977+0.j 0.09955273+0.j
 0.10134654+0.j 0.10516051+0.j 0.09763

In [13]:
# Print actual eigenvalue/eigenvector
print(vals[0], vecs[:,0])

(49.70635338709092+0j) [-0.09318887+0.j -0.10255637+0.j -0.11091952+0.j -0.10177865+0.j
 -0.09711908+0.j -0.10651015+0.j -0.10864416+0.j -0.09964546+0.j
 -0.09689994+0.j -0.09505389+0.j -0.09305672+0.j -0.10299192+0.j
 -0.09589768+0.j -0.10137649+0.j -0.09868125+0.j -0.10214183+0.j
 -0.09932728+0.j -0.10352883+0.j -0.10850549+0.j -0.09190122+0.j
 -0.09411318+0.j -0.10033302+0.j -0.10221064+0.j -0.10409916+0.j
 -0.09758907+0.j -0.0960253 +0.j -0.10927716+0.j -0.0941929 +0.j
 -0.10094037+0.j -0.08709107+0.j -0.09618714+0.j -0.10648278+0.j
 -0.10416788+0.j -0.09638744+0.j -0.1006612 +0.j -0.09329455+0.j
 -0.10323212+0.j -0.09863298+0.j -0.09548265+0.j -0.10228473+0.j
 -0.09859791+0.j -0.09996165+0.j -0.10730829+0.j -0.1009598 +0.j
 -0.09144817+0.j -0.10664134+0.j -0.10529857+0.j -0.09590143+0.j
 -0.10938656+0.j -0.09563107+0.j -0.09183601+0.j -0.10199079+0.j
 -0.10619515+0.j -0.09898924+0.j -0.09721324+0.j -0.10380735+0.j
 -0.09522432+0.j -0.10509251+0.j -0.10498977+0.j -0.09955273+0.j
 -

In [14]:
# Function to run n rounds of RQI and RQIAPP
def runRQI(M, mu, ep, it, n):
  # Save number of iterations to arrays
  its = np.empty(0)
  itsAPP = np.empty(0)

  # Save condition numbers to arrays
  conds = np.empty(0)
  condsAPP = np.empty(0)

  # Save first condition number of each iteration to arrays
  firstConds = np.empty(0)
  firstCondsAPP = np.empty(0)

  # Save last condition number of each iteration to arrays
  lastConds = np.empty(0)
  lastCondsAPP = np.empty(0)

  for i in range(n):
    RQIData = RQI(M, mu, ep, it)
    RQIAPPData = RQIAPP(M, mu, ep, it)

    # Save number of iterations required to the appropriate array
    its = np.append(its, RQIData[2])
    itsAPP = np.append(itsAPP, RQIAPPData[2])

    # Save condition numbers to the appropriate array
    conds = np.append(conds, RQIData[3])
    condsAPP = np.append(condsAPP, RQIAPPData[4])

    # Save first condition number to the appropriate array
    firstConds = np.append(firstConds, RQIData[3][0])
    firstCondsAPP = np.append(firstCondsAPP, RQIAPPData[4][0])

    # Save last condition number to the appropriate array
    lastConds = np.append(lastConds, RQIData[3][-1])
    lastCondsAPP = np.append(lastCondsAPP, RQIAPPData[4][-1])

  # Print iterations required each round
  print('Iterations required each round for RQI:\n', its)
  print('\nIterations required each round for RQIAPP:\n', itsAPP)

  # Get average iterations required by RQI
  index = np.argwhere(its==it)
  its = np.delete(its, index)
  if its.size==0:
    itsAvg = 0
  else:
    itsAvg = np.average(its)

  # Get average condition number for RQI
  condsAvg = np.average(conds)

  # Get average first condition number for RQI
  firstCondsAvg = np.average(firstConds)

  # Get average final condition number for RQI
  lastCondsAvg = np.average(lastConds)

  # Get average iterations required by RQIAPP
  index = np.argwhere(itsAPP==it)
  itsAPP = np.delete(itsAPP, index)
  if itsAPP.size==0:
    itsAPPAvg = 0
  else:
    itsAPPAvg = np.average(itsAPP)

  # Get average condition number for RQIAPP
  condsAPPAvg = np.average(condsAPP)

  # Get average first condition number for RQIAPP
  firstCondsAPPAvg = np.average(firstCondsAPP)

  # Get average final condition number for RQIAPP
  lastCondsAPPAvg = np.average(lastCondsAPP)

  # Print average iterations required and number of errors
  print('\nAverage iterations required for RQI:\n', itsAvg)
  print('\nAverage condition number for RQI:\n', condsAvg)
  print('\nAverage first condition number for RQI:\n', firstCondsAvg)
  print('\nAverage final condition number for RQI:\n', lastCondsAvg)
  print('\nPercentage of times RQI converged:\n', str(its.size/n*100) + '%')
  print('\nAverage iterations required for RQIAPP:\n', itsAPPAvg)
  print('\nAverage condition number for RQIAPP:\n', condsAPPAvg)
  print('\nAverage first condition number for RQIAPP:\n', firstCondsAPPAvg)
  print('\nAverage final condition number for RQIAPP:\n', lastCondsAPPAvg)
  print('\nPercentage of times RQIAPP converged:\n', str(itsAPP.size/n*100) + '%')

In [15]:
# Run 100 rounds with 1e-4
runRQI(M, mu, 1e-4, 20, 100)

Iterations required each round for RQI:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Iterations required each round for RQIAPP:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Average iterations required for RQI:
 2.0

Average condition number for RQI:
 53259.598725103104

Average first condition number for RQI:
 214.22127755663362

Average final condition number for RQI:
 106304.97617264956

Percentage of times RQI converged:
 100.0%

Average iterations required for RQIAP

In [16]:
# Run 100 rounds with 1e-6
runRQI(M, mu, 1e-6, 20, 100)

Iterations required each round for RQI:
 [ 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3. 20.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3. 20.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3. 20.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3. 20.  3.  3.  3.  3.  3.  3.  3.  3.]

Iterations required each round for RQIAPP:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Average iterations required for RQI:
 3.0

Average condition number for RQI:
 1160620510555758.5

Average first condition number for RQI:
 214.22127755663362

Average final condition number for RQI:
 

In [17]:
# Run 100 rounds with 1e-8
runRQI(M, mu, 1e-8, 20, 100)

Iterations required each round for RQI:
 [ 3. 20. 20. 20. 20.  3. 20.  3. 20. 20. 20. 20. 20. 20.  3. 20. 20. 20.
  3. 20.  3. 20.  3. 20. 20. 20. 20.  3. 20. 20.  3. 20. 20. 20. 20. 20.
  3. 20.  3. 20.  3. 20. 20. 20. 20. 20. 20.  3. 20.  3. 20.  3. 20.  3.
  3. 20.  3.  3.  3. 20.  3. 20.  3. 20. 20. 20.  3.  3.  3.  3.  3.  3.
  3.  3. 20.  3. 20. 20. 20. 20. 20.  3. 20. 20. 20. 20.  3.  3. 20. 20.
 20.  3. 20. 20. 20.  3.  3.  3.  3.  3.]

Iterations required each round for RQIAPP:
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 2. 3. 3. 3. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]

Average iterations required for RQI:
 3.0

Average condition number for RQI:
 4679912158100601.0

Average first condition number for RQI:
 214.22127755663362

Average final condition number for RQI:
 

In [18]:
# Run 100 rounds with 1e-10
runRQI(M, mu, 1e-10, 20, 100)

Iterations required each round for RQI:
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]

Iterations required each round for RQIAPP:
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]

Average iterations required for RQI:
 0

Average condition number for RQI:
 5099063765267272.0

Average first condition number for RQI:
 214.22127755663362

Average final condition number for RQI:
 56